# Sales Product Analysis

## Description
This project is dedicated to organizing and analyzing information about product sales analysis to identify trends and generate insights. This project includes an analysis of the top-performing and underperforming products or services, areas, or states with the top sales. Data cleaning are performed using Python, Jupyter Notebooks, and Pandas library.

## Data Overview
Sales Product Data from [Kaggle dataset](https://www.kaggle.com/datasets/knightbearr/sales-product-data?resource=download&select=Sales_September_2019.csv) is used in this analysis. It has 6 fields, such as OrderID, Product, Quantity, Price, Order Data, and Purchase Address, with around 180,000 observations that span 12 months in the year of 2019.

## Research Questions
1. What was the trend for the number of orders and the number of sales? What was the best month for sales?
2. What area (state or city) with the highest number of sales? Which product had the most number of sales?
3. What product with the highest overall number of sales? Were there any specific time it sold during the year?

## Setup and Data Preparation

### Import packages and dataset

In [33]:
# Import packages
import pandas as pd
import numpy as np
import os
import glob

from datetime import datetime

In [15]:
# Read all 12 months dataset

# Read sample data
df = pd.read_csv("./Raw Data/01_Sales_January_2019.csv")
# df.head()

# Append all 12 months data
all_files = glob.glob("./Raw Data/*.csv", recursive=True)

all_data = []

for file in all_files:
    df = pd.read_csv(file, index_col=None, header=0)
    all_data.append(df)

df_raw = pd.concat(all_data, axis=0)

# Save all data to new csv files
df_raw.to_csv("all_months_data.csv", index=False)

In [16]:
# Import newly appended dataset and provide dataset overview
data_name = "./all_months_data.csv"

df_raw = pd.read_csv(data_name)
df_raw.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,194095,Wired Headphones,1,11.99,05/16/19 17:14,"669 2nd St, New York City, NY 10001"
1,194096,AA Batteries (4-pack),1,3.84,05/19/19 14:43,"844 Walnut St, Dallas, TX 75001"
2,194097,27in FHD Monitor,1,149.99,05/24/19 11:36,"164 Madison St, New York City, NY 10001"
3,194098,Wired Headphones,1,11.99,05/02/19 20:40,"622 Meadow St, Dallas, TX 75001"
4,194099,AAA Batteries (4-pack),2,2.99,05/11/19 22:55,"17 Church St, Seattle, WA 98101"


In [18]:
# Print the shape of main dataset
print("Dataset shape:", df_raw.shape)
print('Number of columns :', df_raw.shape[1])
print('Number of records :', df_raw.shape[0], '\n')

print(df_raw.dtypes, '\n')

# Print the column values in the dataframe
print("Column names:")
print(list(df_raw.columns.values))

Dataset shape: (186850, 6)
Number of columns : 6
Number of records : 186850 

Order ID            object
Product             object
Quantity Ordered    object
Price Each          object
Order Date          object
Purchase Address    object
dtype: object 

Column names:
['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date', 'Purchase Address']


### Data Cleaning
This step includes checking missing and duplicate values and changing datatype.

In [19]:
# Check missing values
missing_val = pd.DataFrame(df_raw.isnull().sum())
missing_val.columns = ["missing_count"]
missing_val

,missing_count
Order ID,545
Product,545
Quantity Ordered,545
Price Each,545
Order Date,545
Purchase Address,545


In [20]:
# Drop missing values and recheck
df_raw.dropna(inplace=True)

df_raw.isnull().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

In [23]:
# Check duplicate values
duplicateRows = df_raw[df_raw.duplicated()]
print("Number of duplicate rows: " + str(len(duplicateRows)))

Number of duplicate rows: 618


In [28]:
# Remove duplicate rows and header values
df_raw.drop_duplicates(inplace=True)

df = df_raw[df_raw["Order ID"] != "Order ID"]

Change current datatypes:
- Order ID: integer
- Quantity Ordered: integer
- Price Each: float
- Order Date: date

In [ ]:
# Change datatypes
df["Order ID"] = df["Order ID"].astype('int')
df["Quantity Ordered"] = df["Quantity Ordered"].astype('int')
df["Price Each"] = df["Price Each"].astype('float')
df["Order Date"] = pd.to_datetime(df['Order Date'], format='%m/%d/%y %H:%M')

### Data Transformation
This step includes adding sales (price * qty), as well as extracting month and year from Order Date.

In [ ]:
# Extract month and year from Order Date
df['month'] = df['Order Date'].dt.month
df['month_name'] = df['Order Date'].dt.strftime('%b')
df['year'] = df['Order Date'].dt.year
df['year_month'] = df['Order Date'].dt.strftime('%Y-%m')

In [ ]:
# Create new columns for total sales
df['sales'] = df['Quantity Ordered'] * df['Price Each']

In [42]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,month,month_name,year,year_month,sales
0,194095,Wired Headphones,1,11.99,2019-05-16 17:14:00,"669 2nd St, New York City, NY 10001",5,May,2019,2019-05,11.99
1,194096,AA Batteries (4-pack),1,3.84,2019-05-19 14:43:00,"844 Walnut St, Dallas, TX 75001",5,May,2019,2019-05,3.84
2,194097,27in FHD Monitor,1,149.99,2019-05-24 11:36:00,"164 Madison St, New York City, NY 10001",5,May,2019,2019-05,149.99
3,194098,Wired Headphones,1,11.99,2019-05-02 20:40:00,"622 Meadow St, Dallas, TX 75001",5,May,2019,2019-05,11.99
4,194099,AAA Batteries (4-pack),2,2.99,2019-05-11 22:55:00,"17 Church St, Seattle, WA 98101",5,May,2019,2019-05,5.98


## Analysis

#### Trend of Number of Orders and Sales by Month